In [ ]:
from ..orbdot.star_planet import StarPlanet

planetx = StarPlanet('settings_files/Planet-X_settings.json')

constant_fit = planetx.run_constant_period(['t0', 'P'])

#### 1.1 Output Files
For each model fit in our example the following files are saved:
- `*_summary.txt` : A text summary of the best-fit values and sampling statistics.
- `*_results.json` : The full set of nested sampling outputs.
- `*_random_samples.json`: A set of 300 samples for plotting.
- `*_corner.png` : A corner plot),
- `*_traces.png` : A trace plot).

The summary is a good way to get a quick overview of the results of the model fit.

<details><summary>Summary of constant-period model fit:</summary>

```
Stats
-----
Sampler: nestle
Free parameters: ['t0' 'P']
log(Z) = -189.51807472187025 ± 0.11083889973032876
Run time (s): 6.025493383407593
Num live points: 1000
Evidence tolerance: 0.001
Eff. samples per second: 665

Results
-------
t0 = 2456282.4927388676 ± 7.117870892771849e-05
P = 0.940008751947598 ± 3.7892879371495315e-08
```
</details>

***

#### 1.2 Input Files
In the example above, the creation of the `planetx` object required two files: `'Planet-X_settings.json'` and `'Planet-X_info.json'`.

##### The Settings File
This file is needed to define the directories containing the data (ie. `'Planet-Xb_mid-times.txt'`) via the
`"data_dir"` and `"data_sub"` keys. It also includes important parameters for the nested sampling algorithms such as the
desired sampler (`"nestle"` or `"multinest"`), the prior (descried further below), the number of live points, and the
evidence tolerance. See the other documentation for an explanation of these things.

<details><summary>example:</summary>

```
{"main_save_dir": "Planet-X/Results/",

      "RV_fit": {"sampler":"nestle",
                 "data_dir": "Planet-X/Data/",
                 "data_sub": "_rvs.txt",
                 "n_live_points": 1000,
                 "evidence_tolerance": 0.001},

      "TTV_fit":{"sampler":"nestle",
                  "data_dir": "Planet-X/Data/",
                  "data_sub": "_mid_times.txt",
                  "n_live_points": 1000,
                  "evidence_tolerance": 0.001},
...
```

</details>

##### The System Information File
This file contains the physical characteristics of the star-planet system, including.
- System Parameters: right ascension `"RA"` declination `"DEC"`, distance `"D"`, radial velocity `"v_r"`,
and proper motion `"mu"`
- Star Parameters: mass `"M_s"`, radius `"R_s"`, and rotation period `"P_rot_s"`;
- Planet Parameters: mass `"M_p"`, radius `"R_p"`, orbital period `"P"`, eccentricity `"e"`,
inclination `"i"`, argument of periapse `"w0"`, and RV semi-amplitude `"K"` (In the case of multi-planet systems,
the planet number can be specified as the parameters are held in lists).

<details><summary>example:</summary>

```
{ "_comment1": "System Parameters",
    "Star":"Planet-X",
    "RA":"20h04m00.00s",
    "DEC":"+28d09m19.95s",
    "distance": 160.0,
    "v_r": -15.0,
    "mu_tot": 40.0,

   "_comment2": "Star Characteristics",
    "M_s": 0.90,
    "R_s": 0.95,
    "P_rot": 25,

   "_comment3": "Planet Characteristics",
    "Planet(s)": ["b"],
    "M_p": [300.0],
    "R_p": [11.0],
    "t0": [2456282.493],
    "P": [0.94001],
    "e": [0.0],
    "omega": [0.0],
    "i": [90],
    "K": [253]}
```

</details>

These characteristics are recorded as class attributes, allowing the user to update them easily:

***

#### 1.3 The Parameters
The user enters the free parameters in a model fit as a list of strings that represent different parameters in the
transit timing and/or radial velocity models. There are currently 12 parameters available in the model fitting,
though they may not appear in every model. They fall into three general categories:

###### Orbital Parameters
- `'t0'`: reference transit time [BJD_TDB]
- `'P'`: orbital period in days
- `'e'`: eccentricity
- `'i'`: inclination
- `'w0'`: argument of periapse (A.O.P) in radians (planet or star, depending on model)

###### Time Dependent Parameters
- `'dPdE'`: a constant change of the orbital period in days per orbit
- `'dwdE'`: a constant change of the (A.O.P) (precession) in radians per orbit

###### Radial Velocity Parameters
- `'K'`: radial velocity semi-amplitude in m/s
- `'v0'`: RV zero velocity in m/s (instrument specific)
- `'jit'`: RV jitter in m/s (instrument specific)
- `'dvdt'`: a linear RV acceleration in m/s/day
- `'ddvdt'`: a quadratic RV acceleration in m/s^2/day
***

#### 1.4 The Prior
The `"prior"` is defined in the settings file and is structured as a dictionary with keys for each parameter.

<details><summary>example:</summary>

```
...
  "prior": {"t0":[2456282.5, 0.01],
            "P":[0.94, 0.0001],
            "e":[-8,-1],
            "i":[90,5],
            "w0":[0,6.28318530718],
            "dPdE":[-1e-7, 1e-7],
            "dwdE":[0, 0.001],
            "K":[225, 275],
            "v0":[-10, 10],
            "jit":[-2,2],
            "dvdt":[-1, 1],
            "ddvdt":[-1, 1]}
```
</details>

Each key is a tuple specifying the prior 'bounds' (the meaning of which depend on the type of prior) for transforming
a parameter from the unit hypercube to a normal scale.:
- Gaussian : (mean, std)
- Uniform : (min, max)
- Log-Uniform: (log10(min), log10(max))

Helpful link for explaining the prior
***

### 2. Advanced Model Fitting

##### 2.1 Radial Velocities
Like the transit mid-times, we can run model fits on the radial velocity data (`'Planet-Xb_rvs.txt''`). Let's again
choose a circular orbit with a constant orbital period for our model fit and this time generate a plot to visualize
the results, saved as the file `'*_rv_plot.png'`.

In [ ]:
rv_fit = planetx.run_rv_fit(['t0', 'P', 'K', 'v0'])
planetx.make_rv_plots(show_RV_curve=False)

![RV plot](Planet-X/Results/Planet-X/RV_FITS/solo_rv_plot.png)
***

##### 2.2 More TTV Models
We can do more than just a circular orbit. In addition to the `run_constant_period()` method are `run_orbit_decay()`
and `run_apsidal precession()`, which fit the orbital decay and apsidal precesesion timing models, respectively.
These models are described in more detail in the `Models` docstrings. Let's fit the transit timing data to an orbital
decay model, in which the orbital period will shrink at a constant rate.

In [ ]:
decay_fit = planetx.run_orbital_decay(['t0', 'P', 'dPdE'])

Examining the `'ttv_decay_summary.txt'` file, we see that the Bayesian evidence for the orbital decay model is
`logZ = -60.03 ± 0.14`, whereas the evidence for the constant orbital period was `logZ = -189.52 ± 0.11`, meaning
that orbital decay describes the data far better. We can quantify this by calculating the Baye's factor:

In [ ]:
from ..orbdot.tools.stats import bayes_factor

bayes_factor(decay_fit['stats']['logZ'], constant_fit['stats']['logZ'],
                                                                model_1='Orbital Decay', model_2='Constant Period')

 > Decisive evidence for Orbital Decay vs. Constant Period  (B = 1.7e+56)

Since we have the constant-period solution from earlier, we also visually confirm this result by generating an observed
minus calculated (O-C) plot, which is saved as `'*_O-C_plot.png'`.

In [ ]:
planetx.make_OC_plot(ylims=(-10,10))

![O-C plot](Planet-X/Results/Planet-X/TTV_FITS/ttv_O-C_plot.png)

##### 2.3 Joint Model Fits
We can take this a step further by fitting the radial velocity and transit timing data simultaneously with
the `run_rv_joint_fit` method, the timing model being one of the constant-period, orbital decay, or apsidal precession
models defined in the `Models` class. Let's run two joint RV-timing fits: once with the constant-period model and
once with the orbital decay model. For fun, we will add some Planet-X b eclipse mid-times 'from the literature.' The
results can be seen here(link).

In [ ]:
# run a joint fit of the RV and constant-period models
joint_constant_fit = planetx.run_rv_joint_fit(['t0', 'P', 'K', 'v0', 'jit'],timing_model='constant')
planetx.make_rv_plots(show_RV_curve=False, joint_fit=True, timing_model='constant')

# run a joint fit of the RV and orital decay models
joint_decay_fit = planetx.run_rv_joint_fit(['t0', 'P', 'dPdE', 'K', 'v0', 'jit'],timing_model='decay')
planetx.make_rv_plots(show_RV_curve=False, joint_fit=True, timing_model='decay')

# make an O-C plot
planetx.make_OC_plot(decay=True, joint_fit=True, ylims=(-10,10))

# calculate the Baye's factor
utl.bayes_factor(joint_decay_fit['stats']['logZ'], joint_constant_fit['stats']['logZ'],
                        model_1='Orital Decay', model_2='Constant Period')

Developed by [Simone Hagey](mailto:shagey@phas.ubc.ca)

## References
- [Nestle](http://kbarbary.github.io/nestle) by Kyle Barbary.
- [PyMultiNest](http://johannesbuchner.github.io/PyMultiNest/) by Johannes Buchner.